In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ava-v2/ava_val_excluded_timestamps_v2.2.csv
/kaggle/input/ava-v2/ava_action_list_v2.2_for_activitynet_2019.pbtxt
/kaggle/input/ava-v2/ava_train_v2.2.csv
/kaggle/input/ava-v2/ava_train_excluded_timestamps_v2.2.csv
/kaggle/input/ava-v2/ava_included_timestamps_v2.2.txt
/kaggle/input/ava-v2/ava_action_list_v2.2.pbtxt
/kaggle/input/ava-v2/ava_val_v2.2.csv
/kaggle/input/ava-v2/ava_test_excluded_timestamps_v2.2.csv
/kaggle/input/ava-v2/ava_test_v2.2.txt


In [107]:
# Importing necessary libraries for the project
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import shutil

# Define paths to the dataset and directories
BASE_PATH = '/kaggle/input/ava-v2/'  # Update the base path for your dataset location
TRAIN_CSV_PATH = os.path.join(BASE_PATH, 'ava_train_v2.2.csv')
VAL_CSV_PATH = os.path.join(BASE_PATH, 'ava_val_v2.2.csv')
TEST_CSV_PATH = os.path.join(BASE_PATH, 'ava_test_v2.2.csv')
ACTION_LIST_PATH = os.path.join(BASE_PATH, 'ava_action_list_v2.2.pbtxt')  # Action list file path

# Dataset directories (adjust as per your data organization)
TRAIN_IMAGES_DIR = '/kaggle/input/ava-v2-images/train_images/'
VAL_IMAGES_DIR = '/kaggle/input/ava-v2-images/val_images/'
TEST_IMAGES_DIR = '/kaggle/input/ava-v2-images/test_images/'

# Output directories (for saving models, logs, etc.)
OUTPUT_DIR = '/kaggle/working/'



In [21]:
# Read the contents of the action list file to inspect its structure
with open("/kaggle/input/ava-v2/ava_action_list_v2.2.pbtxt") as f:
    action_list_content = f.read()

# Print the contents to understand its structure
print(action_list_content[:])  # Print the first 2000 characters for inspection


label {
  name: "bend/bow (at the waist)"
  label_id: 1
  label_type: PERSON_MOVEMENT
}
label {
  name: "crawl"
  label_id: 2
  label_type: PERSON_MOVEMENT
}
label {
  name: "crouch/kneel"
  label_id: 3
  label_type: PERSON_MOVEMENT
}
label {
  name: "dance"
  label_id: 4
  label_type: PERSON_MOVEMENT
}
label {
  name: "fall down"
  label_id: 5
  label_type: PERSON_MOVEMENT
}
label {
  name: "get up"
  label_id: 6
  label_type: PERSON_MOVEMENT
}
label {
  name: "jump/leap"
  label_id: 7
  label_type: PERSON_MOVEMENT
}
label {
  name: "lie/sleep"
  label_id: 8
  label_type: PERSON_MOVEMENT
}
label {
  name: "martial art"
  label_id: 9
  label_type: PERSON_MOVEMENT
}
label {
  name: "run/jog"
  label_id: 10
  label_type: PERSON_MOVEMENT
}
label {
  name: "sit"
  label_id: 11
  label_type: PERSON_MOVEMENT
}
label {
  name: "stand"
  label_id: 12
  label_type: PERSON_MOVEMENT
}
label {
  name: "swim"
  label_id: 13
  label_type: PERSON_MOVEMENT
}
label {
  name: "walk"
  label_id: 14
  lab

In [28]:
# Define the target actions related to violence and crime
target_actions = {
    "fight/hit (a person)",
    "grab (a person)",
    "push (another person)",
    "kick (a person)"
}

# Initialize an empty dictionary to hold action ID to action name mappings
action_list = {}

# Read the action list file
with open("/kaggle/input/ava-v2/ava_action_list_v2.2.pbtxt") as f:
    action_lines = f.readlines()

# Process each line to extract action names and IDs
for line in action_lines:
    if 'name:' in line:
        action_name = line.strip().split(": ")[1].strip('\"')
    if 'label_id:' in line:
        action_id = int(line.strip().split(": ")[1])
        
        # Check if the action is in our target actions list
        if action_name in target_actions:
            action_list[action_id] = action_name

# Display the filtered action list (only target actions)
print("Filtered Target Actions and Their IDs:")
for action_id, action_name in action_list.items():
    print(f"Action ID: {action_id}, Action Name: {action_name}")


Filtered Target Actions and Their IDs:
Action ID: 64, Action Name: fight/hit (a person)
Action ID: 66, Action Name: grab (a person)
Action ID: 71, Action Name: kick (a person)
Action ID: 76, Action Name: push (another person)


In [109]:
import pandas as pd

# Define the column names
column_names =  ['video_id', 'middle_frame_timestamp', 'x1', 'y1', 'x2', 'y2', 'action_id', 'person_id']

# Load the dataset with the column names
dataset = pd.read_csv('/kaggle/input/ava-v2/ava_train_v2.2.csv', names= column_names)

# Print the first few rows to confirm the columns are correctly added
print(dataset.head())


      video_id  middle_frame_timestamp     x1     y1     x2     y2  action_id  \
0  -5KQ66BBWC4                     902  0.077  0.151  0.283  0.811         80   
1  -5KQ66BBWC4                     902  0.077  0.151  0.283  0.811          9   
2  -5KQ66BBWC4                     902  0.226  0.032  0.366  0.497         12   
3  -5KQ66BBWC4                     902  0.226  0.032  0.366  0.497         17   
4  -5KQ66BBWC4                     902  0.226  0.032  0.366  0.497         80   

   person_id  
0          1  
1          1  
2          0  
3          0  
4          0  


In [110]:
dataset[4000:5000]

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id,person_id
4000,-FaXLcSFjUI,1719,0.118,0.046,0.581,0.981,12,223
4001,-FaXLcSFjUI,1719,0.118,0.046,0.581,0.981,79,223
4002,-FaXLcSFjUI,1719,0.445,0.131,0.628,0.671,12,224
4003,-FaXLcSFjUI,1719,0.445,0.131,0.628,0.671,17,224
4004,-FaXLcSFjUI,1719,0.445,0.131,0.628,0.671,80,224
...,...,...,...,...,...,...,...,...
4995,-IELREHX_js,1123,0.397,0.355,0.969,0.974,8,92
4996,-IELREHX_js,1124,0.411,0.235,0.965,0.965,8,92
4997,-IELREHX_js,1125,0.061,0.039,0.405,0.791,12,94
4998,-IELREHX_js,1127,0.071,0.054,0.414,0.776,12,94


In [112]:
# Print the first few rows of the datdataset.head()
df.to_csv('train.csv', index=False)
dataset.head()

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id,person_id
0,-5KQ66BBWC4,902,0.077,0.151,0.283,0.811,80,1
1,-5KQ66BBWC4,902,0.077,0.151,0.283,0.811,9,1
2,-5KQ66BBWC4,902,0.226,0.032,0.366,0.497,12,0
3,-5KQ66BBWC4,902,0.226,0.032,0.366,0.497,17,0
4,-5KQ66BBWC4,902,0.226,0.032,0.366,0.497,80,0


In [101]:
# Check for missing values
print(dataset.isnull().sum())

# Optional: Drop rows with missing values
dataset = dataset.dropna()

video_id                  0
middle_frame_timestamp    0
person_box                0
action_id                 0
person_id                 0
dtype: int64


In [102]:
print(dataset.columns)

Index(['video_id', 'middle_frame_timestamp', 'person_box', 'action_id',
       'person_id'],
      dtype='object')


In [118]:
# Define target action IDs for violence/crime detection
target_action_ids = [64, 66, 71, 76]  # Fight, Grab, Kick, Push

# Load the training dataset
train_dataset = dataset

# Filter the training dataset based on action IDs
filtered_train_data = []

for _, row in train_dataset.iterrows():
    action_id = row['action_id']
    if action_id in target_action_ids:
        filtered_train_data.append({
            'video_id': row['video_id'],
            'middle_frame_timestamp': row['middle_frame_timestamp'],
            'x1': row['x1'],
            'y1': row['y1'],
            'x2': row['x2'],
            'y2': row['y2'],
            'action_id': action_id
        })

# Convert filtered data into DataFrame (for further use in model training)
filtered_train_df = pd.DataFrame(filtered_train_data)

# Save the filtered train dataset to CSV for later use
filtered_train_df.to_csv(os.path.join(OUTPUT_DIR, 'filtered_train_data.csv'), index=False)

# Display filtered train data
print("Filtered Train Dataset based on Target Actions:")
print(filtered_train_df.head())


Filtered Train Dataset based on Target Actions:
      video_id  middle_frame_timestamp     x1     y1     x2     y2  action_id
0  -5KQ66BBWC4                     922  0.206  0.168  0.805  0.970         64
1  -5KQ66BBWC4                     923  0.622  0.133  0.807  0.985         64
2  -5KQ66BBWC4                     925  0.285  0.233  0.541  0.930         64
3  -5KQ66BBWC4                     925  0.547  0.145  0.848  0.999         64
4  -5KQ66BBWC4                     926  0.378  0.222  0.887  0.959         64


In [119]:
# Save filtered data to CSV file
filtered_train_df.to_csv('/kaggle/working/filtered_train_data.csv', index=False)  # Save to CSV without index column
print("Filtered data has been saved to 'filtered_train_data.csv'")

Filtered data has been saved to 'filtered_train_data.csv'


In [116]:
# Load the validation dataset
val_dataset = pd.read_csv(VAL_CSV_PATH, names=column_names)  # Update column_names if needed

df.to_csv('val.csv', index=False)
val_dataset.head()

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id,person_id
0,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,12,1
1,1j20qq1JyX4,902,0.002,0.118,0.714,0.977,79,1
2,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,12,0
3,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,17,0
4,1j20qq1JyX4,902,0.444,0.054,0.992,0.990,79,0


In [120]:

# Filter the validation dataset based on action IDs
filtered_val_data = []

for _, row in val_dataset.iterrows():
    action_id = row['action_id']
    if action_id in target_action_ids:
        filtered_val_data.append({
            'video_id': row['video_id'],
            'middle_frame_timestamp': row['middle_frame_timestamp'],
            'x1': row['x1'],
            'y1': row['y1'],
            'x2': row['x2'],
            'y2': row['y2'],
            'action_id': action_id
        })

# Convert filtered data into DataFrame (for further use in validation)
filtered_val_df = pd.DataFrame(filtered_val_data)

# Save the filtered validation dataset to CSV for later use
filtered_val_df.to_csv(os.path.join(OUTPUT_DIR, 'filtered_val_data.csv'), index=False)

# Display filtered validation data
print("Filtered Validation Dataset based on Target Actions:")
print(filtered_val_df.head())

Filtered Validation Dataset based on Target Actions:
      video_id  middle_frame_timestamp     x1     y1     x2     y2  action_id
0  2PpxiG0WU18                     930  0.524  0.382  0.682  0.970         66
1  2PpxiG0WU18                     936  0.582  0.307  0.761  0.953         66
2  2PpxiG0WU18                     946  0.551  0.251  0.839  0.980         66
3  2PpxiG0WU18                     950  0.010  0.162  0.514  0.926         66
4  2PpxiG0WU18                    1146  0.209  0.071  0.971  0.958         66


In [121]:
filtered_val_df.to_csv('/kaggle/working/filtered_val_data.csv', index=False)  # Save to CSV without index column
print("Filtered data has been saved to 'filtered_val_data.csv'")

Filtered data has been saved to 'filtered_val_data.csv'
